<a href="https://colab.research.google.com/github/vtsimoes/class_victor_dataset/blob/main/G_Transforma_Dataset_VICTOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Representação Computacional dos documentos do DATASET VICTOR para TF-IDF, Word2Vec e FastText.


In [ ]:
#Load required libraries
import os
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('rslp')
from collections import Counter
import matplotlib.pyplot as plt
from matplotlib import pylab
import time
import pickle
from google.colab import drive
import string
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import KeyedVectors
from gensim.models.word2vec import LineSentence
from gensim.models.callbacks import CallbackAny2Vec

Funções para salvar os dados pré-processados

In [ ]:
#Montando o google drive para obter os dados
def save_pickle(data_to_save,path='',filename='file.pkl'):
  open_file = open(os.path.join(path,filename), "wb")
  pickle.dump(data_to_save, open_file)
  open_file.close()

def load_pickle(path='',filename='file.pkl'):
  open_file = open(path + filename, "rb")
  pkl_file = pickle.load(open_file)
  open_file.close()
  return pkl_file

In [ ]:
path = '' #path com os dados pré-processados
path_transformados = '' #path para salvar os documentos transformados
tf_idf = 'TF-IDF'
word2vec = 'Word2Vec'
path_models = 'Models'
path_w2v_nilc = 'skip_nilc_word2vec' # path com os vetores pré-treinados do NILC/USP
#Montando o google drive para obter os dados
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
path_dados_word2vec = os.path.join(path_transformados,word2vec,path_models)
path_dados_word2vec

##TF-IDF

Para aplicação no SVM e Regressão Logística, a geração do modelo TF-IDF vai incluir apenas os dados de treino

In [ ]:
doc_train = load_pickle(path=path,filename='train_processed_concat_pages_sw_lemma.pkl')

In [ ]:
def TF_IDFTransform(data,model=None,max_vocab=70000,gram=(1,1)):
  sentences = data.body

  if model:
    sparse_tfidf = model.transform(sentences)
    tf_idf_model = model
  else:
    tf_idf_model = TfidfVectorizer(ngram_range=gram,sublinear_tf=True,max_features=max_vocab,min_df=2,max_df=0.5)
    tf_idf_model.fit(sentences)
    sparse_tfidf = tf_idf_model.transform(sentences)
  return sparse_tfidf,tf_idf_model

In [ ]:
train_tfidf_values,tfidf_model = TF_IDFTransform(doc_train,model=None,max_vocab=70000,gram=(1,1)) #para a formação de bigram usar gram=(1,2)
train_tfidf= {'vector':train_tfidf_values,'labels':doc_train.document_type.values,'file_name':doc_train.file_name.values,'pages':doc_train.pages.values}
del(doc_train)

Empregando o modelo criado com o conjunto de validação e teste.

---



In [ ]:
doc_test = load_pickle(path=path,filename='test_processed_concat_pages_lemma.pkl')
doc_valid = load_pickle(path=path,filename='valid_processed_concat_pages_lemma.pkl')

In [ ]:
test_tfidf_values,_ = TF_IDFTransform(doc_test,model=tfidf_model,max_vocab=70000)
test_tfidf = {'vector':test_tfidf_values,'labels':doc_test.document_type.values,'file_name':doc_test.file_name.values,'pages':doc_test.pages.values}
del(doc_test)

In [ ]:
valid_tfidf_values,_ = TF_IDFTransform(doc_valid,model=tfidf_model,max_vocab=70000)
valid_tfidf = {'vector':valid_tfidf_values,'labels':doc_valid.document_type.values,'file_name':doc_valid.file_name.values,'pages':doc_valid.pages.values}
del(doc_valid)

Salvando os dados e o modelo para utilização futura

In [ ]:
save_pickle(train_tfidf,path=path_transformados,filename='train_tfidf_small_70K_O100.pkl') #small indica que o TF-IDF sofreu um maior pré-processamento
save_pickle(test_tfidf,path=path_transformados,filename='test_tfidf_small_70K_O100.pkl')
save_pickle(valid_tfidf,path=path_transformados,filename='valid_tfidf_small_70K_O100.pkl')
save_pickle(tfidf_model,path=path_transformados+tf_idf+'/'+path_models,filename='model_tfidf_small_70K_O100.pkl')

In [ ]:
del(test_tfidf)
del(train_tfidf)
del(tfidf_model)
del(valid_tfidf)

Gerando TF-IDF para o conjunto NO

In [ ]:
doc_train = load_pickle(path=path,filename='train_processed_concat_pages_lemma.pkl')
doc_train = doc_train.loc[doc_train.document_type != 'outros']
doc_valid = load_pickle(path=path,filename='valid_processed_concat_pages_lemma.pkl')
doc_valid = doc_valid.loc[doc_valid.document_type != 'outros']
doc_test = load_pickle(path=path,filename='test_processed_concat_pages_lemma.pkl')
doc_test = doc_test.loc[doc_test.document_type != 'outros']

In [ ]:
train_tfidf_NO,tfidf_model_NO = TF_IDFTransform(doc_train,model=None,max_vocab=70000)

In [ ]:
test_tfidf_NO,_ = TF_IDFTransform(doc_test,model=tfidf_model_NO,max_vocab=70000)
valid_tfidf_NO,_ = TF_IDFTransform(doc_valid,model=tfidf_model_NO,max_vocab=70000)

In [ ]:
train_tfidf = {'vector':train_tfidf_NO,'labels':doc_train.document_type.values,'file_name':doc_train.file_name.values,'pages':doc_train.pages.values}
test_tfidf = {'vector':test_tfidf_NO,'labels':doc_test.document_type.values,'file_name':doc_test.file_name.values,'pages':doc_test.pages.values}
valid_tfidf = {'vector':valid_tfidf_NO,'labels':doc_valid.document_type.values,'file_name':doc_valid.file_name.values,'pages':doc_valid.pages.values}

In [ ]:
save_pickle(train_tfidf,path=path_transformados,filename='train_tfidf_small_70K_NO.pkl')
save_pickle(test_tfidf,path=path_transformados,filename='test_tfidf_small_70K_NO.pkl')
save_pickle(valid_tfidf,path=path_transformados,filename='valid_tfidf_small_70K_NO.pkl')
save_pickle(tfidf_model_NO,path=path_transformados+tf_idf+'/'+path_models,filename='model_tfidf_small_70K_NO.pkl')

In [ ]:
del(train_tfidf_NO)
del(tfidf_model_NO,test_tfidf_NO,valid_tfidf_NO)
del(train_tfidf,test_tfidf,valid_tfidf)

## Gerando TF-IDF para dataset O10

In [ ]:
doc_train = load_pickle(path=path,filename='train_processed_concat_pages_lemma.pkl')
doc_valid = load_pickle(path=path,filename='valid_processed_concat_pages_lemma.pkl')
train_O10_filenames = load_pickle(path=path,filename='filenames_train_O10.pkl')
valid_O10_filenames = load_pickle(path=path,filename='filenames_valid_O10.pkl')

In [ ]:
doc_train_O10 = doc_train.loc[doc_train.file_name.isin(train_O10_filenames)]
doc_train_O10 = pd.concat([doc_train_O10,doc_train.loc[doc_train.document_type != 'outros']], ignore_index=True, sort=False)
doc_valid_O10 = doc_valid.loc[doc_valid.file_name.isin(valid_O10_filenames)]
doc_valid_O10 = pd.concat([doc_valid_O10,doc_valid.loc[doc_valid.document_type != 'outros']], ignore_index=True, sort=False)

In [ ]:
del(doc_train,doc_valid)

In [ ]:
model_tf_idf = load_pickle(path=path_transformados+'TF-IDF/Models/',filename='model_tfidf_small_70K_O100.pkl')

In [ ]:
train_tfidf_values_O10,_ = TF_IDFTransform(doc_train_O10,model=model_tf_idf,max_vocab=70000)
train_tfidf_O10 = {'vector':train_tfidf_values_O10,'labels':doc_train_O10.document_type.values,'file_name':doc_train_O10.file_name.values,'pages':doc_train_O10.pages.values}

In [ ]:
valid_tfidf_values_O10,_ = TF_IDFTransform(doc_valid_O10,model=model_tf_idf,max_vocab=70000)
valid_tfidf_O10 = {'vector':valid_tfidf_values_O10,'labels':doc_valid_O10.document_type.values,'file_name':doc_valid_O10.file_name.values,'pages':doc_valid_O10.pages.values}

In [ ]:
save_pickle(train_tfidf_O10 ,path=os.path.join(path_transformados,tf_idf),filename='train_tfidf_small_70K_O10.pkl')
save_pickle(valid_tfidf_O10,path=os.path.join(path_transformados,tf_idf),filename='valid_tfidf_small_70K_O10.pkl')

#Representação com Word2Vec

##Geração de vetores Word2Vec a partir dos dados de Treino


In [ ]:
path_models_word2vec = os.path.join(path_transformados,word2vec,path_models,path_w2v_nilc) #carregando os vetores pré-treinados do NILC

In [ ]:
from gensim.models import Word2Vec
loss_word2vec = []

In [ ]:
class callback(CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''

    def __init__(self):
        self.epoch = 0
        self.loss_to_be_subed = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        model.save(os.path.join(path_dados_word2vec,f'epoch-{self.epoch}-w2v-600-model.model'))
        vecs = model.wv
        vecs.save(os.path.join(path_dados_word2vec,f'epoch-{self.epoch}-w2v-600-model.vector'))
        loss_now = loss - self.loss_to_be_subed
        loss_word2vec.append(loss_now)
        self.loss_to_be_subed = loss
        print('Loss after epoch {}: {}'.format(self.epoch, loss_now))
        self.epoch += 1

In [ ]:
doc_train = load_pickle(path=path,filename='train_processed_concat_pages.pkl')
docs = doc_train.body.apply(lambda x: x.split())

In [ ]:
tamanho = 600 #tamanho do vetor de representação da palavra artigo de Word2Vec em português (NILC) e MPPR
window = 10 #tamanho da janela empregada no processo de treinamento, artigo do mikolov e MPPR
iter = 10 #número de épocas a ser definido com eraly stopping
sg = 1 #define o skip-gram como arquitetura
alpha=0.025 #taxa de aprendizado (Mikolov)
max_vocab = 70000
min_count = 5 #(MPPR)

In [ ]:
#Treinamento dos vetores com Word2Vec
w2v_model = Word2Vec(docs, vector_size=tamanho, workers=4, window=window, epochs=iter,sg=sg,alpha=alpha, min_count=min_count,max_vocab_size=max_vocab,compute_loss=True,callbacks=[callback()] )

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:


ypoints = np.log(loss_word2vec)
xpoints = range(1,len(loss_word2vec)+1)

plt.plot(xpoints, ypoints)
plt.ylabel('log(Loss)')
plt.xlabel('Época')
plt.grid()
plt.suptitle('Logarítmo do Erro (Loss) por época de treinamento dos vetores especializados Word2Vec')
plt.show()


Salvando o modelo e os dados transformados

In [ ]:
w2v_model.save(path_transformados+path_models+'word2vec_10_epochs_600_70K.model')
vectors = w2v_model.wv
vectors.save(path_transformados+path_models+'word2vec_10_epochs_600_70K.vector')
save_pickle(loss_word2vec,path=path_transformados+word2vec+'/'+path_models,filename='word2vec_loss_600_epochs-0-10.pkl')

In [ ]:
save_pickle(loss_word2vec,path=path_transformados+path_models,filename='word2vec_loss_600_100O.pkl')
del(w2v_model)

##Gerando representação dos documentos com Word2Vec


In [ ]:
doc_train = load_pickle(path=path,filename='train_processed_concat_pages.pkl')
doc_valid = load_pickle(path=path,filename='valid_processed_concat_pages.pkl')
doc_test = load_pickle(path=path,filename='test_processed_concat_pages.pkl')

In [ ]:
seq_len = 1530
size_ebd = 600

In [ ]:
# Método para representação computacional a partir dos vetores de embedding, a variável model pode ser do modelo genérico ou especializado
def doc2vector(data,model,tipo='mean',seq_len=500,tfidf=False,size_ebd=300,tfidf_model=None,sparse_tfidf=None):
  doc_vecs = []
  labels = []
  files = []
  i=0
  for i, row in data.iterrows():
    #emb = [model[w] for text in data.body for w in text[0:seq_len] if w in model]
    words = row.body.split()[0:seq_len]
    #print(words)
    words = [w for w in words if w in model]
    emb = [model[w] for w in words]
    labels.append(row.document_type)
    if len(emb) == 0:
        doc_vecs.append(np.zeros(size_ebd, dtype=np.float32))
    else:
      doc_vecs.append(np.mean(emb,axis=0))
      #vecs.append(emb[0:500])
    #i = 1 + i
    files.append(row.file_name)
  #return doc_vecs,vecs
  data_dict = {'w2v':np.array(doc_vecs),'labels':np.array(labels),'filename':np.array(files)}
  return data_dict

In [ ]:
#Unzip no arquivo baixado do NILC, somente se ainda não foi descompactado
from zipfile import ZipFile
zip_nilc_600 = os.path.join(path_dados_word2vec_nilc,'skip_s600.zip')
with ZipFile(zip_nilc_600, 'r') as zObject:
  zObject.extractall(path=path_dados_word2vec_nilc)

### Representação Com Word2Vec pré-treinado do NILC (600) ou especializado

Carregando os vetores do modelos Word2Vec

In [ ]:
from gensim.models import Word2Vec
vectors = KeyedVectors.load_word2vec_format(os.path.join(path_dados_word2vec_nilc,'skip_s600.txt')) #usando o modelo genérico do NILC

#### Transformação dos Dados de Treino

In [ ]:
#vector_size = model_w2vec_nilc.vector_size
vector_size = tamanho
w2v_train = doc2vector(doc_train,vectors,tipo='mean',seq_len=seq_len,tfidf=False,size_ebd=vector_size,tfidf_model=None,sparse_tfidf=None)
save_pickle(w2v_train,path=os.path.join(path_transformados,word2vec),filename='train_w2v_600_1530_specialized_O100.pkl') #OK

In [ ]:
w2v_train_NO = doc2vector(doc_train.loc[doc_train.document_type != 'outros'],vectors,tipo='mean',seq_len=seq_len,tfidf=False,size_ebd=vector_size,tfidf_model=None,sparse_tfidf=None)
save_pickle(w2v_train_NO,path=os.path.join(path_transformados,word2vec),filename='train_w2v_600_1530_specialized_NO.pkl') #OK

Gerando Vetor com filenames aletórios para o conjunto de treino O10.

In [ ]:
import random
def index_sample(data,percent):
  data_o = data.loc[data.document_type == 'outros']
  filenames_o = data_o.file_name.values
  sampled = random.sample(list(filenames_o),int(0.10*len(filenames_o)))
  return sampled

In [ ]:
filenames_O10 = load_pickle(path=path,filename='filenames_train_O10.pkl')

In [ ]:
#filenames_O10 = index_sample(doc_train,0.10)
#save_pickle(filenames_O10,path=path,filename='filenames_train_O10.pkl')
doc_train_O10 = doc_train.loc[doc_train.file_name.isin(filenames_O10)]
doc_train_O10 = pd.concat([doc_train_O10,doc_train.loc[doc_train.document_type != 'outros']], ignore_index=True, sort=False)

In [ ]:
w2v_train_O10 = doc2vector(doc_train_O10,vectors,tipo='mean',seq_len=seq_len,tfidf=False,size_ebd=vector_size,tfidf_model=None,sparse_tfidf=None)
save_pickle(w2v_train_O10,path=os.path.join(path_transformados,word2vec),filename='train_w2v_600_1530_specialized_O10.pkl') #OK

In [ ]:
del(doc_train,doc_train_O10)
del(w2v_train_O10,w2v_train_NO,w2v_train)

#### Transformação dos dados de Validação

In [ ]:
w2v_valid = doc2vector(doc_valid,vectors,tipo='mean',seq_len=seq_len,tfidf=False,size_ebd=vector_size,tfidf_model=None,sparse_tfidf=None)
save_pickle(w2v_valid,path=os.path.join(path_transformados,word2vec),filename='valid_w2v_600_1530_specialized_O100.pkl') #OK

In [ ]:
w2v_valid_NO = doc2vector(doc_valid.loc[doc_valid.document_type != 'outros'],vectors,tipo='mean',seq_len=seq_len,tfidf=False,size_ebd=vector_size,tfidf_model=None,sparse_tfidf=None)
save_pickle(w2v_valid_NO,path=os.path.join(path_transformados,word2vec),filename='valid_w2v_600_1530_specialized_NO.pkl')

Definição do conjunto aleatório de documentos O10 para validação

In [ ]:
#filenames_O10 = index_sample(doc_valid,0.10)
#save_pickle(filenames_O10,path=path,filename='filenames_valid_O10.pkl')
filenames_O10 = load_pickle(path=path,filename='filenames_valid_O10.pkl')
doc_valid_O10 = doc_valid.loc[doc_valid.file_name.isin(filenames_O10)]
doc_valid_O10 = pd.concat([doc_valid_O10,doc_valid.loc[doc_valid.document_type != 'outros']], ignore_index=True, sort=False)

In [ ]:
w2v_valid_O10 = doc2vector(doc_valid_O10,vectors,tipo='mean',seq_len=seq_len,tfidf=False,size_ebd=vector_size,tfidf_model=None,sparse_tfidf=None)
save_pickle(w2v_valid_O10,path=os.path.join(path_transformados,word2vec),filename='valid_w2v_600_1530_specialized_O10.pkl') #OK

In [ ]:
del(doc_valid,doc_valid_O10,w2v_valid,w2v_valid_NO,w2v_valid_O10)

#### Transformação dos dados de Teste

In [ ]:
w2v_test = doc2vector(doc_test,vectors,tipo='mean',seq_len=seq_len,tfidf=False,size_ebd=vector_size,tfidf_model=None,sparse_tfidf=None)
save_pickle(w2v_test ,path=os.path.join(path_transformados,word2vec),filename='test_w2v_600_1530_specialized_O100.pkl') #OK

In [ ]:
w2v_test_NO = doc2vector(doc_test.loc[doc_test.document_type != 'outros'],vectors,tipo='mean',seq_len=seq_len,tfidf=False,size_ebd=vector_size,tfidf_model=None,sparse_tfidf=None)
save_pickle(w2v_test_NO,path=os.path.join(path_transformados,word2vec),filename='test_w2v_600_1530_specialized_NO.pkl') #OK

In [ ]:
del(w2v_test,w2v_test_NO,doc_test)

# FastText

In [ ]:
from gensim.models import FastText
from gensim.models import KeyedVectors

## Transformando os Dados com vetores de embedding pré-treinados (genéricos) do NILC/USP


In [ ]:
doc_train = load_pickle(path=path,filename='train_processed_concat_pages.pkl')
doc_valid = load_pickle(path=path,filename='valid_processed_concat_pages.pkl')
doc_test = load_pickle(path=path,filename='test_processed_concat_pages.pkl')

In [ ]:
fasttext = 'fasttext/'
path_fasttext = 'skip_s600'
path_dados_fasttext = os.path.join(path_transformados,fasttext,path_models)

In [ ]:
from zipfile import ZipFile
zip_nilc_600 = os.path.join(path_dados_fasttext ,'skip_s600.zip')
with ZipFile(zip_nilc_600, 'r') as zObject:
  zObject.extractall(path=path_dados_fasttext)

In [ ]:
path_dados_fasttext_nilc = os.path.join(path_transformados,fasttext,path_models)
model_fast_text_nilc = KeyedVectors.load_word2vec_format(os.path.join(path_dados_fasttext,'skip_s600.txt'))

### Dados de Treino

In [ ]:
vector_size = model_fast_text_nilc.vector_size

In [ ]:
w2v_train = doc2vector(doc_train,model_fast_text_nilc,tipo='mean',seq_len=seq_len,tfidf=False,size_ebd=vector_size,tfidf_model=None,sparse_tfidf=None)
save_pickle(w2v_train,path=os.path.join(path_transformados,fasttext),filename='train_fasttext_600_1530_specialized_O100.pkl') #OK

In [ ]:
w2v_train_NO = doc2vector(doc_train.loc[doc_train.document_type != 'outros'],model_fast_text_nilc,tipo='mean',seq_len=seq_len,tfidf=False,size_ebd=vector_size,tfidf_model=None,sparse_tfidf=None)
save_pickle(w2v_train_NO,path=os.path.join(path_transformados,fasttext),filename='train_fasttext_600_1530_specialized_NO.pkl') #OK

In [ ]:
filenames_O10 = load_pickle(path=path,filename='filenames_train_O10.pkl')
doc_train_O10 = doc_train.loc[doc_train.file_name.isin(filenames_O10)]
doc_train_O10 = pd.concat([doc_train_O10,doc_train.loc[doc_train.document_type != 'outros']], ignore_index=True, sort=False)

In [ ]:
w2v_train_O10 = doc2vector(doc_train_O10,model_fast_text_nilc,tipo='mean',seq_len=seq_len,tfidf=False,size_ebd=vector_size,tfidf_model=None,sparse_tfidf=None)
save_pickle(w2v_train_O10,path=os.path.join(path_transformados,fasttext),filename='train_fasttext_600_1530_specialized_O10.pkl') #OK

### Dados de Validação

In [ ]:
w2v_valid = doc2vector(doc_valid,model_fast_text_nilc,tipo='mean',seq_len=seq_len,tfidf=False,size_ebd=vector_size,tfidf_model=None,sparse_tfidf=None)
save_pickle(w2v_valid,path=os.path.join(path_transformados,fasttext),filename='valid_fasttext_600_1530_specialized_O100.pkl') #OK

In [ ]:
w2v_valid_NO = doc2vector(doc_valid.loc[doc_valid.document_type != 'outros'],model_fast_text_nilc,tipo='mean',seq_len=seq_len,tfidf=False,size_ebd=vector_size,tfidf_model=None,sparse_tfidf=None)
save_pickle(w2v_valid_NO,path=os.path.join(path_transformados,fasttext),filename='valid_fasttext_600_1530_specialized_NO.pkl')

In [ ]:
filenames_O10  = load_pickle(path=path,filename='filenames_valid_O10.pkl')
doc_valid_O10 = doc_valid.loc[doc_valid.file_name.isin(filenames_O10)]
doc_valid_O10 = pd.concat([doc_valid_O10,doc_valid.loc[doc_valid.document_type != 'outros']], ignore_index=True, sort=False)

In [ ]:
w2v_valid_O10 = doc2vector(doc_valid_O10,model_fast_text_nilc,tipo='mean',seq_len=seq_len,tfidf=False,size_ebd=vector_size,tfidf_model=None,sparse_tfidf=None)
save_pickle(w2v_valid_O10,path=os.path.join(path_transformados,fasttext),filename='valid_fasttext_600_1530_specialized_O10.pkl') #OK

### Dados de Teste

In [ ]:
w2v_test = doc2vector(doc_test,model_fast_text_nilc,tipo='mean',seq_len=seq_len,tfidf=False,size_ebd=vector_size,tfidf_model=None,sparse_tfidf=None)
save_pickle(w2v_test ,path=os.path.join(path_transformados,fasttext),filename='test_fasttext_600_1530_specialized_O100.pkl') #OK

In [ ]:
w2v_test_NO = doc2vector(doc_test.loc[doc_test.document_type != 'outros'],model_fast_text_nilc,tipo='mean',seq_len=seq_len,tfidf=False,size_ebd=vector_size,tfidf_model=None,sparse_tfidf=None)
save_pickle(w2v_test_NO,path=os.path.join(path_transformados,fasttext),filename='test_fasttext_600_1530_specialized_NO.pkl') #OK

In [ ]:
del(doc_train,doc_valid,doc_test)

## Gerando Vetores Especializados FastText a partir dos dados de treino

In [ ]:
class callback_fast(CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''

    def __init__(self):
        self.epoch = 0
        self.loss_to_be_subed = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        model.save(os.path.join(path_dados_fasttext,f'epoch-{self.epoch}-fast-600-model.model'))
        vecs = model.wv
        vecs.save(os.path.join(path_dados_fasttext,f'epoch-{self.epoch}-fast-600-model.vector'))
        loss_now = loss - self.loss_to_be_subed
        loss_word2vec.append(loss_now)
        self.loss_to_be_subed = loss
        print('Loss after epoch {}: {}'.format(self.epoch, loss_now))
        self.epoch += 1

In [ ]:
doc_train = load_pickle(path=path,filename='train_processed_concat_pages.pkl')
docs = doc_train.body.apply(lambda x: x.split())

In [ ]:
tamanho = 600 #tamanho do vetor de representação da palavra artigo de Word2Vec em português (NILC) e MPPR
window = 10 #tamanho da janela empregada no processo de treinamento, artigo do mikolov e MPPR
iter = 5 #número de épocas obtida de Mikolov
sg = 1 #define o skip-gram como arquitetura
alpha=0.025 #taxa de aprendizado (Mikolov)
max_vocab = 70000
min_count = 5 #(MPPR)
min_n=3
max_n=6

In [ ]:
time_start = time.time()
fast_model = FastText(docs, vector_size=tamanho, workers=4, window=window, epochs=iter,sg=sg,alpha=alpha, min_count=min_count,max_vocab_size=max_vocab)
time_end = time.time() - time_start
time_end

In [ ]:
import tempfile
open_file = open(path_transformados+fasttext+'/'+path_models+'/'+'fasttext_10_epochs_600.model', "wb")
fast_model.save(open_file, separately=[])
#fname = get_tmpfile(path_transformados+path_models+'/'+'fasttext_10_epochs_600.model')
#fast_model.save(fname)

In [ ]:
vectors = fast_model.wv
vectors.save(path_transformados+path_models+'/'+'fasttext_10_epochs_600.vector')
#save_pickle(loss_word2vec,path=path_transformados+path_models,filename='fasttext_loss_600_epochs-1-10.pkl')

In [ ]:
open_file = open(path_transformados+fasttext+'/'+path_models+'/'+'fasttext_10_epochs_600.vector', "wb")
vectors.save(open_file)
#Obs: Para gerar a representação dos documentos com o FastText especializado, usar a mesma sequência acima empregada para o FastText genérico, apenas substituindo o arquivo carregado